In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def max_length_text(df):
    max_length = 0
    for i in range(df.shape[0]):
        length = np.size(word_tokenize(df.at[i, 'text']))
        if length > max_length: max_length = length
    return max_length

In [3]:
def text_normalization(data):
    data['text'] = data['text'].apply(lambda x: x.lower())
    data['text'] = data['text'].apply((lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x)))

In [4]:
def remove_stop_words(data, language):
    stopwords = nltk.corpus.stopwords.words(language)
    for i in range(data.shape[0]):
        data.at[i, 'text'] = [word for word in nltk.word_tokenize(data.at[i, 'text']) if word not in stopwords]
        data.at[i, 'text'] = ' '.join(data.at[i, 'text'])

In [5]:
def apply_stemming(data, language):
    stemmer = SnowballStemmer(language)
    for i in range(data.shape[0]):
         data.at[i, 'text'] = (' '.join([stemmer.stem(word) for word in data.at[i, 'text'].split()]))

get_matrix representation

In [14]:
def get_matrix(data, representation, vocabulary_length, stemming, remove_stopwords, language):

    df = data.copy(deep = True)
    
    text_normalization(df) # Text normalization
    
    # Stop_words
    if remove_stopwords:
        remove_stop_words(df, language)
    
    # Stemming
    if stemming:
        apply_stemming(df, language)
    
    # Word representation
    if representation == 'BoW':
        #count_vectorizer = CountVectorizer(max_df = 0.9, max_features = vocabulary_length, min_df = 0.1)
        count_vectorizer = CountVectorizer(max_features = vocabulary_length)
        matrix = count_vectorizer.fit_transform(df.text)
        
    elif representation == 'tf-idf':
        #tfidf_vectorizer = TfidfVectorizer(max_df = 0.9, max_features = vocabulary_length, min_df = 0.1, use_idf=True)
        tfidf_vectorizer = TfidfVectorizer(max_features = vocabulary_length, use_idf=True)
        matrix = tfidf_vectorizer.fit_transform(df.text)
    
    return matrix, df

In [7]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
...,...,...
2566,"Recuperamos la historia de Aleixandra, la jove...",0
2567,"Reproches, tensión y sinceridad: la comida en ...",0
2568,"RT @ElMundoOpinion: ""PSOE, PP, Ciudadanos y Vo...",0
2569,Rusia cita al embajador español por unas decla...,0


In [8]:
print(dataset.at[0, 'text'])

RAE INCLUIRÁ LA PALABRA "LADY" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE "MUJER PROBLEMÁTICA"
España.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término "Lady" como una nueva palabra en el diccionario del idioma español.
Darío señaló que "Lady" servirá para definir a una "mujer problemática" o a una "mujer que causa problemas", y mencionó que esta palabra será una de las pocas que también se utilizan en el idioma inglés pero que en castellano tiene un significado diferente:
"Son contadas las palabras del idioma inglés que se utilizan en el español pero que tienen otro significado. Con la globalización las personas han comenzado a adoptar términos anglosajones pero los utilizan con su significado real, sin embargo en este caso la expresión Lady no significará lo mismo que en su idioma original ("dama" en inglés) sino que se usará para definir a una mujer que es problemáti

In [15]:
matrix, df = get_matrix(data = dataset, representation = 'tf-idf', vocabulary_length = 50000, stemming = True, remove_stopwords = True, language = 'spanish')
print(df.at[0, 'text'])
print(matrix.shape)
print(matrix.toarray())

rae inclu palabr lady diccionari idiom español definicion muj problemat españ president real academi español rae dari villanuev inform conferent prens part proxim mes inclu termin lady nuev palabr diccionari idiom español dari señal lady serv defin muj problemat muj caus problem mencion palabr poc utiliz idiom ingles castellan signific diferent cont palabr idiom ingles utiliz español signific globaliz person comenz adopt termin anglosajon utiliz signific real embarg cas expresion lady signific mism idiom original dam ingles sin usar defin muj problemat acostumbr caus problem alborot gent podr dec lady femin caus algun escandal agres provoqu algun tip problem termin dej consider palabr exclus idiom ingles tras inclu diccionari lengu español pas ser palabr oficial castellan signific distint confes villanuev present medi definicion oficial aparec diccionari señal siguient lady anglosajon ænglisc part difficilis problemat ferox agres numb adj f muj exces problemat numb adj f muj caus probl

In [16]:
print(matrix[0])

  (0, 49)	0.14697856906977055
  (0, 10)	0.14009449219471973
  (0, 28)	0.12950632140890075
  (0, 22)	0.12470913139466505
  (0, 23)	0.14714405009170628
  (0, 46)	0.12292365957582499
  (0, 38)	0.2554840835555722
  (0, 27)	0.4097822095419885
  (0, 19)	0.13855926345420705
  (0, 41)	0.12083262348490409
  (0, 30)	0.11641040938316255
  (0, 48)	0.28757915280751595
  (0, 8)	0.1421265973698731
  (0, 33)	0.14952640450270696
  (0, 24)	0.14009449219471973
  (0, 5)	0.1298448906057435
  (0, 31)	0.1376086415204816
  (0, 42)	0.4360682801209263
  (0, 26)	0.12491191963451485
  (0, 21)	0.29264521670536775
  (0, 29)	0.22232659072879243
  (0, 15)	0.138696705284583
  (0, 35)	0.2615205531005691
